# STL-10 dataset (using pretrained weights)

> The STL-10 dataset is an image recognition dataset for developing unsupervised feature learning, deep learning, self-taught learning algorithms. It is inspired by the CIFAR-10 dataset but with some modifications. In particular, each class has fewer labeled training examples than in CIFAR-10, but a very large set of unlabeled examples is provided to learn image models prior to supervised training. The primary challenge is to make use of the unlabeled data (which comes from a similar but different distribution from the labeled data) to build a useful prior. We also expect that the higher resolution of this dataset (96x96) will make it a challenging benchmark for developing more scalable unsupervised learning methods.


The STL-10 dataset is interesting as it basically provides you two ways of solving it: 
- Use a pretrained network
- Learn the image features, for example using a autoencoder to learn what features are important

In this notebook I will provide a basic example of using a pretrained network. In this case this approach is a little bit of cheating. The STL-10 dataset takes data from imagenet, so we can take a network pretrained on imagenet and immediately get a good performance. 


In [ ]:
import utils.plot_utils 

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models


In [60]:
transform=torchvision.transforms.Compose([
                           torchvision.transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_dataset = torchvision.datasets.STL10('./data', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.STL10('./data', split='test', transform=transform, download=True)

BATCH_SIZE=64
TEST_BATCH_SIZE=64

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE)


Files already downloaded and verified
Files already downloaded and verified


In [61]:
def train(model, device, train_loader, optimizer, epoch, log_every=5):
    model.train()
    l = nn.CrossEntropyLoss()
    losses = list()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = l(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_every == 0:
            losses.append(loss.item())
            print(loss.item())
    return losses

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    l = nn.CrossEntropyLoss()

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += l(output, target).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [62]:
def get_pretrained_network():
    resnet = models.resnet18(pretrained=True)
    
    # delete the last fc layer and spatial pooling layer
    modules = list(resnet.children())[:-1] 
    
    # We use a smaller image size, so replace the avg pool 
    modules[-1] = torch.nn.AvgPool2d(kernel_size=3, stride=1, padding=0) 
    
    # 'Rebuild' the model
    resnet = nn.Sequential(*modules)

#     # Do not overwrite the pretrained weights
#     for param in resnet.parameters():
#         param.requires_grad = False
    return resnet

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.pretrained = get_pretrained_network()
        self.fc = nn.Linear(512, 10)
    def forward(self, x):
        x = self.pretrained(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc(x))
        return x
    

In [63]:
EPOCHS=10
LEARNING_RATE=0.01

model = Network().to(device)
#optimizer = optim.Adam(model.parameters()) # Feel free to use a different optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(EPOCHS):
    losses = train(model, device, train_loader, optimizer, epoch)
    utils.plot_utils.plot_loss(losses, epoch)
    test(model, device, test_loader)


2.435403347015381
2.3799679279327393
2.291201591491699
2.1396875381469727


KeyboardInterrupt: 